In [ ]:
!pip install -U efficientnet
from efficientnet.tfkeras import EfficientNetB1

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from sklearn.model_selection import train_test_split

####Spectrogram creators####
from scipy import signal as sig
!pip install nnAudio
from nnAudio.Spectrogram import CQT1992v2
############################
 
import matplotlib.pyplot as plt
import torch
from glob import glob

In [ ]:
root = '../input/g2net-gravitational-wave-detection/'
files_paths = glob(root + '/train/*/*/*/*')
train_labels = pd.read_csv(root+'/training_labels.csv')
ids_from_files_paths = [path.split('/')[-1].split('.')[0] for path in files_paths]
ids_paths = pd.DataFrame({
    'id':ids_from_files_paths,
    'path':files_paths
})
train_desc = pd.merge(left = train_labels, right = ids_paths, on = 'id')
display(train_desc)

In [ ]:
def cqt_1992(data_from_record):
#     for i in range(3):
#         data_from_record[i]=data_from_record[i]/np.max(data_from_record)
    data_combined = np.hstack(data_from_record)
    data_combined = np.hstack(data_from_record)/np.max(data_from_record)
    spectrogram_creator = CQT1992v2(
        sr=2048, hop_length=64, fmin=20, fmax=1024, verbose=False
    )
    img = spectrogram_creator(torch.from_numpy(data_combined).float())[0]
    ret = np.stack([img,img,img],axis = -1)
    return ret
func = cqt_1992
data_from_record = np.load(train_desc.loc[0, 'path'])
img = cqt_1992(data_from_record)
s_w, s_h, s_c = cqt_1992(data_from_record).shape
print('s_w:'+str(s_w))
print('s_h:'+str(s_h))
print('s_c:'+str(s_c))
plt.imshow(img)

In [ ]:
# def cqt_comb(data_from_record):
#     spectrogram_creator = CQT1992v2(
#         sr=2048, hop_length=64, fmin=20, fmax=1024, 
#         bins_per_octave=12, norm=1, window='hann', center=True, pad_mode='reflect', trainable=False, 
#         output_format='Magnitude', verbose=False
#     )
#     img_1 = spectrogram_creator(torch.from_numpy(data_from_record[0]).float())[0]
#     print(img_1.shape)
#     img_2 = spectrogram_creator(torch.from_numpy(data_from_record[1]).float())[0]
#     img_3 = spectrogram_creator(torch.from_numpy(data_from_record[2]).float())[0]
#     img = torch.stack((img_1, img_2, img_3),0)
# #     ret = np.stack([img,img,img],axis = -1)
#     ret = img
#     return ret
# func = cqt_comb
# data_from_record = np.load(train_desc.loc[0, 'path'])
# img = cqt_comb(data_from_record)
# print(cqt_comb(data_from_record).shape)
# print('s_w:'+str(s_w))
# print('s_h:'+str(s_h))
# print('s_c:'+str(s_c))
# plt.figure(figsize=(10, 6), dpi=100)
# plt.imshow(img)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, path, list_IDs, data, batch_size):
            self.path = path
            self.list_IDs = list_IDs
            self.data = data
            self.batch_size = batch_size
            self.indexes = np.arange(len(self.list_IDs))
    
    def __len__(self):
        len_ = int(len(self.list_IDs)/self.batch_size)
        if len_ * self.batch_size < len(self.list_IDs):
            len_ += 1
        return len_
    
    def __data_generation(self, list_IDs_temp):
        X = np.zeros((self.batch_size, s_w, s_h, s_c))
        y = np.zeros((self.batch_size, 1))
        for i, ID in enumerate(list_IDs_temp):
            id_ = self.data.loc[ID, 'id']
            file = id_+".npy"
            path_in = '/'.join([self.path, id_[0], id_[1], id_[2]]) + '/'
            data_array = np.load(path_in+file)
            X[i, ] = func(data_array)
            y[i, ] = self.data.loc[ID, 'target']
        return X, y
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index+1) * self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

In [ ]:
sample_submission = pd.read_csv(root+'/sample_submission.csv')
train_ids = train_labels['id'].values
y = train_labels['target'].values
train_indices, validation_indices = train_test_split(list(train_labels.index))
print(len(train_indices))
print(len(validation_indices))
train_gen = DataGenerator(root+'/train/', train_indices, train_labels, 64)
valid_gen = DataGenerator(root+'/train/', validation_indices, train_labels, 64)

In [ ]:
model = tf.keras.Sequential()
# first pair
model.add(tf.keras.layers.Conv2D(filters=16,
                 kernel_size=3,
                 input_shape=(s_w, s_h, s_c),
                 activation='relu',
))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
# second pair
model.add(tf.keras.layers.Conv2D(filters=32,
                 kernel_size=3,
                 input_shape=(s_w, s_h, s_c),
                 activation='relu',
))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
# third pair
model.add(tf.keras.layers.Conv2D(filters=64,
                 kernel_size=3,
                 input_shape=(s_w, s_h, s_c),
                 activation='relu',
))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(
    512,
    activation="relu"
))
model.add(tf.keras.layers.Dense(
    128,
    activation="relu"
)) 
model.add(tf.keras.layers.Dense(
    32,
    activation="relu"
)) 

model.add(tf.keras.layers.Dense(
    1,
    activation="sigmoid"
))
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4),
    loss = 'binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TruePositives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalseNegatives()]
)
model.summary()

In [ ]:
history = model.fit_generator(generator=train_gen, validation_data=valid_gen, epochs = 1, workers=-1)

In [ ]:
model.save('/kaggle/working/trained_cnn_2.h5')

In [ ]:
test_indices = list(sample_submission.index)
test_gen = DataGenerator( root+'/test/', test_indices, sample_submission, 512)
predicted_test_seq_keras = model.predict_generator(test_gen, verbose=1)
sample_submission['target'] = predicted_test_seq_keras[:len(sample_submission)]

sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission['target'] = predicted_test_seq_keras[:len(sample_submission)]

sample_submission.to_csv('submission.csv', index=False)